In [1]:
import numpy as np
from astropy import units as au 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column


plt.style.use('paper.mplstyle')

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
filters = [
            ['IC~5332', 'F658N WFC3'],
            ['NGC~0628', 'F658N ACS'],
            ['NGC~1087', 'F657N WFC3'],
            ['NGC~1300', 'F658N ACS'],
            ['NGC~1365', 'F657N WFC3'],
            ['NGC~1385', 'F657N WFC3'],
            ['NGC~1433', 'F657N WFC3'],
            ['NGC~1512', 'F658N WFC3'],
            ['NGC~1566', 'F658N WFC3'],
            ['NGC~1672', 'F658N ACS'],
            ['NGC~2835', 'F658N WFC3'],
            ['NGC~3351', 'F658N WFC3'],
            ['NGC~3627', 'F658N WFC3'],
            ['NGC~4254', 'F657N WFC3'],
            ['NGC~4303', 'F658N WFC3'],
            ['NGC~4321', 'F657N WFC3'],
            ['NGC~4535', 'F657N WFC3'],
            ['NGC~5068', 'F658N WFC3'],
            ['NGC~7496', 'F657N WFC3'],
            ]

filters = np.array(filters)

table_filters = QTable(filters, names=('gal_name', 'filter'))
table_filters

gal_name,filter
str10,str10
IC~5332,F658N WFC3
NGC~0628,F658N ACS
NGC~1087,F657N WFC3
NGC~1300,F658N ACS
NGC~1365,F657N WFC3
NGC~1385,F657N WFC3
NGC~1433,F657N WFC3
NGC~1512,F658N WFC3
NGC~1566,F658N WFC3


In [4]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha_10pc_fixednoise.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha_10pc_fixednoise.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/dat

In [5]:
output_stats = []

galaxies = []
stds = []
stds_as2 = []


for file_maps in files_maps: 

    # Get galaxy names
    galaxy = file_maps.split(root_dir)[-1].split('/')[0]
    galaxies += [galaxy]
    print('Galaxy: ', galaxy)

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    data = hstha_hdu.data
    header = hstha_hdu.header

    # Get pixel size 
    if 'CDELT1' in  header and 'CDELT2' in  header:
        pix = np.array([abs(header['CDELT1']), abs(header['CDELT2'])]).mean() * au.degree
    else:
        pix = 1

    # fallback if these are placeholders
    if pix.value == 1:
        if 'CD1_1' in  header and 'CD2_2' in  header:
            pix = np.array([abs(header['CD1_1']), abs(header['CD2_2'])]).mean() * au.degree
            print('\t [INFO] Pixel scale taken as CD1_1, CD2_2')
        elif 'PC1_1' in  header and 'PC2_2' in  header:
            pix = np.array([abs(header['PC1_1']), abs(header['PC2_2'])]).mean() * au.degree
            print('\t [INFO] Pixel scale taken as PC1_1, PC2_2')
    else:
        print('\t [INFO] Pixel scale taken as CDELT1, CDELT2')
    pix = pix.to('arcsec')
    print(f'\t [INFO] Pixel scale -- > {pix:.2f}')

    # Get RMS for whole map... 
    # std = stats.mad_std(data, ignore_nan=True)  # Get noise 
    std = stats.mad_std(data, ignore_nan=True)
    std = stats.mad_std(data[data < std*5], ignore_nan=True)

    std_as2 = std / pix.value**2

    print(f'\t MAD STD (pix) = {std:.2f}')
    print(f'\t MAD STD (as2) = {std_as2:.2f}')

    stds += [std]   
    stds_as2 += [std_as2]

stds = np.array(stds)
stds_as2 = np.array(stds_as2)

Galaxy:  ngc3627
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 105.75
	 MAD STD (as2) = 67368.10
Galaxy:  ngc4254
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 102.86
	 MAD STD (as2) = 65524.88
Galaxy:  ngc1512
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 100.03
	 MAD STD (as2) = 63721.46
Galaxy:  ngc1433
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 100.49
	 MAD STD (as2) = 64019.52
Galaxy:  ngc5068
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 103.53
	 MAD STD (as2) = 65951.40
Galaxy:  ngc4321
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- > 0.04 arcsec
	 MAD STD (pix) = 102.49
	 MAD STD (as2) = 65293.29
Galaxy:  ngc1365n
	 [INFO] Pixel scale taken as CDELT1, CDELT2
	 [INFO] Pixel scale -- >

In [6]:
# Get into a astropy table the output of the stats of the maps to .tex
gal_name = ['']*len(galaxies)
for i, gal in enumerate(galaxies):

    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628'
    if '1365N' in gal:
        gal_split = gal.split('1365N')
        gal = gal_split[0] + '1365'
    if '2835S' in gal:
        gal_split = gal.split('2835S')
        gal = gal_split[0] + '2835'

    gal_name[i] = gal

table1 = Column(gal_name, name='gal_name')
table2 = QTable(np.array([stds, stds_as2/1e3]).T, names=('std', 'std_as2'))

table = hstack([table1, table2])
table.sort('gal_name')
table

table.write('./tabs/galmap_props_10pc.fits', overwrite=True)

In [7]:
table_out = join(table, table_filters, keys='gal_name')
table_out = table_out['gal_name', 'filter', 'std_as2']

formats={
        'gal_name': '%s',
        'filter': '%s',
        'std_as2': '%0.1f'}

table_out.write('./tabs/galmap_props_10pc.tex', formats=formats, overwrite=True)

In [8]:
table.sort('std')
table

gal_name,std,std_as2
str8,float64,float64
NGC~1512,100.0264000996273,63.72145688153326
NGC~4303,100.17511051349673,63.816192313536995
NGC~1365,100.18990576879004,63.82561757748016
NGC~1433,100.49428308124249,64.01952004494713
NGC~1300,100.7433139226321,64.1781642429386
NGC~7496,100.77915961378234,64.20099961100888
NGC~1566,100.77930666142669,64.20109328717956
IC~5332,100.91359509479231,64.28664124859648
NGC~3351,101.19648082845478,64.46685258342428
